In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [5]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('../LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

# convert N to 0 and Y to Yes
x = dict.fromkeys(['n','N'],0)
df = df.replace(x)
x = dict.fromkeys(['y','Y'],0)
df = df.replace(x)

# dictionary for home ownership
ownership_num = {
    'MORTGAGE': 0,
    'RENT': 1,
    'OWN': 2,
    'ANY': 3
}

df["home_ownership_int"] = df["home_ownership"].apply(lambda x: ownership_num[x])
df = df.drop("home_ownership",axis=1)

# dictionary for verification
verification_num = {
    'Not Verified': 0,
    'Source Verified': 1,
    'Verified': 2
}

df["verification_status_int"] = df["verification_status"].apply(lambda x: verification_num[x])
df = df.drop("verification_status",axis=1)

# dictionary for issue date
month_num = {
    'Jan-2019': 0,
    'Feb-2019': 1,
    'Mar-2019': 2,
    'Apr-2019': 3,
    'May-2019': 4,
}

df["issue_d_int"] = df["issue_d"].apply(lambda x: month_num[x])
df = df.drop("issue_d",axis=1)

df["next_pymnt_d_int"] = df["next_pymnt_d"].apply(lambda x: month_num[x])
df = df.drop("next_pymnt_d",axis=1)

# encode for application type and list status
df = pd.get_dummies(df, columns=["application_type","initial_list_status"])

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,hardship_flag,debt_settlement_flag,home_ownership_int,verification_status_int,issue_d_int,next_pymnt_d_int,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w
0,10500.0,0.1719,375.35,66000.0,low_risk,0,27.24,0.0,0.0,8.0,...,0,0,1,1,2,4,1,0,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,0,20.23,0.0,0.0,17.0,...,0,0,0,2,2,4,1,0,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,0,24.26,0.0,0.0,8.0,...,0,0,0,2,2,4,1,0,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,0,31.44,0.0,1.0,10.0,...,0,0,1,2,2,4,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,0,18.76,0.0,1.0,14.0,...,0,0,0,0,2,4,1,0,0,1


In [7]:
# get info for encoding
df.select_dtypes(['object'])

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk
...,...
68812,low_risk
68813,low_risk
68814,low_risk
68815,low_risk


# Split the Data into Training and Testing

In [8]:
# Create our features
X = df.drop('loan_status',axis=1)

# Create our target
y = df['loan_status']

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,hardship_flag,debt_settlement_flag,home_ownership_int,verification_status_int,issue_d_int,next_pymnt_d_int,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.0,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.0,0.0,0.599009,0.669994,0.726172,3.616839,0.860340,0.139660,0.123879,0.876121
std,10277.348590,0.048130,288.062432,1.155800e+05,0.0,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.0,0.0,0.713731,0.719105,0.743862,0.486161,0.346637,0.346637,0.329446,0.329446
min,1000.000000,0.060000,30.890000,4.000000e+01,0.0,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.0,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.000000,1.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,0.0,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.0,0.0,0.000000,1.000000,1.000000,4.000000,1.000000,0.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,0.0,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.0,0.0,1.000000,1.000000,1.000000,4.000000,1.000000,0.000000,0.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,0.0,999.000000,18.000000,5.000000,72.000000,4.000000,...,0.0,0.0,3.000000,2.000000,2.000000,4.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
Counter(y_train)

Counter({'low_risk': 51352, 'high_risk': 260})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [16]:
# Resample the training data with the RandomOversampler then run balanced random forest
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

from imblearn.ensemble import BalancedRandomForestClassifier
# create a random forest
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_resampled, y_resampled)

BalancedRandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                               criterion='gini', max_depth=None,
                               max_features='auto', max_leaf_nodes=None,
                               max_samples=None, min_impurity_decrease=0.0,
                               min_samples_leaf=2, min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=100,
                               n_jobs=None, oob_score=False, random_state=1,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=False)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6778104113032863

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   31,    56],
       [   12, 17106]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.72      0.36      1.00      0.48      0.60      0.33        87
   low_risk       1.00      1.00      0.36      1.00      0.60      0.38     17118

avg / total       1.00      1.00      0.36      1.00      0.60      0.38     17205



In [21]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08858297017591785, 'total_rec_prncp'),
 (0.08117273654835154, 'total_pymnt'),
 (0.06094828247072772, 'last_pymnt_amnt'),
 (0.0603351731119759, 'total_pymnt_inv'),
 (0.05950883323666644, 'total_rec_int'),
 (0.05014493867514421, 'issue_d_int'),
 (0.027305368100494533, 'int_rate'),
 (0.02004050786075487, 'out_prncp_inv'),
 (0.01985442413619274, 'installment'),
 (0.01791954085748638, 'annual_inc'),
 (0.01781170763155525, 'out_prncp'),
 (0.016793128014902518, 'dti'),
 (0.016698931759078137, 'avg_cur_bal'),
 (0.016632423005637845, 'max_bal_bc'),
 (0.014742798633370404, 'mths_since_recent_inq'),
 (0.013958774842580759, 'bc_util'),
 (0.013623766581403698, 'revol_bal'),
 (0.013209211011820952, 'mo_sin_old_il_acct'),
 (0.013043475090034912, 'total_rev_hi_lim'),
 (0.0130261993850166, 'il_util'),
 (0.012990711056075764, 'tot_hi_cred_lim'),
 (0.012962358651025917, 'bc_open_to_buy'),
 (0.012565012132214837, 'total_bc_limit'),
 (0.01255949376900573, 'mo_sin_old_rev_tl_op'),
 (0.012217079447238094

### Easy Ensemble AdaBoost Classifier

In [22]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
# create a adaboost
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee_model.fit(X_resampled, y_resampled)

EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=None,
                       random_state=1, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [23]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = ee_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9285376151741864

In [24]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   77,    10],
       [  479, 16639]], dtype=int64)

In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.14      0.89      0.97      0.24      0.93      0.85        87
   low_risk       1.00      0.97      0.89      0.99      0.93      0.87     17118

avg / total       1.00      0.97      0.89      0.98      0.93      0.87     17205



Easy Ensemble w/ Less Estimators

In [26]:
# Train the Classifier with less estimators
from imblearn.ensemble import EasyEnsembleClassifier
# create a adaboost
ee_model = EasyEnsembleClassifier(n_estimators=10, random_state=1)
ee_model.fit(X_resampled, y_resampled)

EasyEnsembleClassifier(base_estimator=None, n_estimators=10, n_jobs=None,
                       random_state=1, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [27]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = ee_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9285376151741864

In [28]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   77,    10],
       [  479, 16639]], dtype=int64)

In [29]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.14      0.89      0.97      0.24      0.93      0.85        87
   low_risk       1.00      0.97      0.89      0.99      0.93      0.87     17118

avg / total       1.00      0.97      0.89      0.98      0.93      0.87     17205

